In [1]:
!pip install google-api-python-client

  Obtaining dependency information for google-api-python-client from https://files.pythonhosted.org/packages/e6/53/4e109982df222687dea9e736499c69076d7d00699a14470ee281c5cf9eed/google_api_python_client-2.144.0-py2.py3-none-any.whl.metadata
  Obtaining dependency information for httplib2<1.dev0,>=0.19.0 from https://files.pythonhosted.org/packages/a8/6c/d2fbdaaa5959339d53ba38e94c123e4e84b8fbc4b84beb0e70d7c1608486/httplib2-0.22.0-py3-none-any.whl.metadata
  Obtaining dependency information for google-auth-httplib2<1.0.0,>=0.2.0 from https://files.pythonhosted.org/packages/be/8a/fe34d2f3f9470a27b01c9e76226965863f153d5fbe276f83608562e49c04/google_auth_httplib2-0.2.0-py2.py3-none-any.whl.metadata
  Obtaining dependency information for google-api-core!=2.0.*,!=2.1.*,!=2.2.*,!=2.3.0,<3.0.0.dev0,>=1.31.5 from https://files.pythonhosted.org/packages/79/53/2e340a6ed897fa2bdd6c1bf166b98c047fbb648463dfd2b209ca7d501984/google_api_core-2.19.2-py3-none-any.whl.metadata
  Obtaining dependency informati

In [13]:
import os 
import pandas as pd
from dotenv import load_dotenv
from googleapiclient.discovery import build
load_dotenv()

True

In [12]:
#importing dataset
data = pd.read_csv("../data/ytUK.csv")

In [15]:
# Function for fetching necessary data from youtube

def youtubeData(df):
    api_key = os.getenv('API_KEY')
    youtube = build('youtube', 'v3', developerKey=api_key)

    # Initializing empty lists to store the results 
    subscriber_counts = []
    total_views = []
    video_counts = []
    
    # Looping through the DataFrame, extracting the channel Od and making API requests
    for index, row in df.iterrows():
        # Extract the channel ID after the '@'
        channel_info = row['Name']
        channel_id = channel_info.split('@')[1] if '@' in channel_info else None
        
        if channel_id:
            #Define the parameters for the API request
            request = youtube.channels().list(
                part='snippet,contentDetails,statistics',
                id=channel_id
            )
            try:
                response = request.execute() # Executing the API request
            
                #  Extracting information from the response
                if "items" in response and len(response["items"]) > 0:
                    channel_info = response["items"][0]

                    # Subscriber count, total views and video count
                    subscriber_count = channel_info["statistics"].get("subscriberCount", "Hidden")
                    total_view = channel_info["statistics"].get("viewCount", "N/A")
                    video_count = channel_info["statistics"].get("videoCount", "N/A")
                    
                    # Appending the extracted data to the lists
                    subscriber_counts.append(subscriber_count)
                    total_views.append(total_view)
                    video_counts.append(video_count)
                else:
                    print(f"Channel ID {channel_id}: No data found")
                    subscriber_counts.append("N/A")
                    total_views.append("N/A")
                    video_counts.append("N/A")
            except Exception as e:
                print(f"Error fetching Data for channel ID {channel_id}: {str(e)}")
                subscriber_counts.append("N/A")
                total_views.append("N/A")
                video_counts.append("N/A")
        else:
            # If channel ID is not found, append NA to lists
            print(f"No valid channel ID found for row {index}")
            subscriber_counts.append("N/A")
            total_views.append("N/A")
            video_counts.append("N/A")
            
    # Adding the extracted data to the DataFrame
    data['Subscribers'] = subscriber_counts
    data['TotalViews'] = total_views
    data['TotalVideos'] = video_counts
    
    # Saving the updated Dataframe in csv format
    df.to_csv('../data/Channel_statistics2.csv', index=False)
        

In [16]:
youtubeData(data)

Channel ID sidemen: No data found
Channel ID juliusdein: No data found
Channel ID liverpoolfc: No data found
Channel ID sirhcchris2010: No data found
Channel ID thedadlab: No data found
